#  AI - CA (DZ Day)

### We use uninformed and informed search algorithm (BFS, IDS, A*) in this project.

Map of problem is implemented by `Map` class that has `Nodes`. Each node has adjacency list and 3 identifiers which are `is_eater`, `is_recipe` and `is_hard`.

In [1]:
import time
import heapq
from dataclasses import dataclass
from copy import deepcopy
from functools import partial
from typing import Callable, Any


@dataclass
class Node:
    def __init__(self) -> None:
        self.is_eater = False
        self.is_recipe = False
        self.is_hard = False
        self.adj = []

    @property
    def is_special(self):
        return self.is_eater or self.is_recipe
    

@dataclass
class Map:
    def __init__(self, v):
        self.v = v
        self.nodes = [Node() for _ in range(v)]
        
    def add_edge(self, a, b):
        self.nodes[a].adj.append(b)
        self.nodes[b].adj.append(a)

map: Map


### Problem's states are implemented by `State` class. Each state has properties:

1- Current location -> `loc`

2- Nodes' visitin and timer status -> `nodes`

3- Eaters that all of their recipes are found and their nodes are visited -> `satisfied_eaters`

4- Recipes that their nodes are visited -> `found_recipes`

5- Cost of crossed path -> `time`

6- Crossed path until current state -> `path`
\
\
\
To distinguish states we hash `satisfied_eaters`, `found_recipes` & `loc`.

In [2]:
@dataclass
class State:
    def __init__(self, v):
        self.loc: int
        self.nodes = [{'visited':False, 'timer':1} for _ in range(v)]
        self.satisfied_eaters = set()
        self.found_recipes = set()
        self.time = 0
        self.path = []

    def _tuple(self):
        return (tuple(self.satisfied_eaters), tuple(self.found_recipes), self.loc)

    def __hash__(self):
        return hash(self._tuple())


`eaters_to_recipes` is a dict that specifies recipes of each eater.

In [3]:
eaters_to_recipes: dict[list[int]]

Following function read input file and return initial state.

In [4]:
def read_file(file_name):
    with open(file_name) as f:
        n, m = [int(x) for x in f.readline().split()]

        initial_state = State(n)

        global map, eaters_to_recipes
        map = Map(n)
        eaters_to_recipes = dict()

        for _ in range(int(m)):
            a, b = f.readline().split()
            map.add_edge(int(a) - 1, int(b) - 1)

        h = int(f.readline())
        hard_nodes = [int(x) for x in f.readline().split()]
        for v in hard_nodes:
            map.nodes[v - 1].is_hard = True
        
        s = int(f.readline())
        for i in range(s):
            line = [int(x) for x in f.readline().split()]

            eater = line[0]
            map.nodes[eater - 1].is_eater = True

            for r in line[2:]:
                map.nodes[r - 1].is_recipe = True
                eaters_to_recipes[eater - 1] = eaters_to_recipes.get(eater - 1, [])
                eaters_to_recipes[eater - 1].append(r - 1)
        
        v = int(f.readline())
        initial_state.loc = v - 1
        initial_state.path.append(v)

        return initial_state


To sort states by different scale, we implement a heap structure that named `StateHeap`. It uses predicator callable to sort states.

In [5]:
class StateHeap:
    def __init__(self, key=lambda x:x):
        self.key = key
        self.index = 0
        self._data = []

    def push(self, item):
        heapq.heappush(self._data, (self.key(item), self.index, item))
        self.index += 1

    def pop(self):
        return heapq.heappop(self._data)[2]

    def __bool__(self):
        return bool(self._data)


`is_satisfied` checks if all of eater's recipes are found or not.

Action:
`move` function gets a state and new location and after creating new state, returns it.

`is_goal` checks if current state is goal or not.

In [6]:
def is_satisfied(eater_recipes, found_recipes):
    for recipe in eater_recipes:
        if recipe not in found_recipes:
            return False
    return True


def move(cur_state: State, new_loc):
    next_state = deepcopy(cur_state)
    
    next_state.loc = new_loc
    next_state.nodes[new_loc]['visited'] = True
    next_state.time += next_state.nodes[new_loc]['timer']

    if map.nodes[new_loc].is_hard:
        next_state.nodes[new_loc]['timer'] += 1
    if map.nodes[new_loc].is_recipe:
        next_state.found_recipes.add(new_loc)
    if map.nodes[new_loc].is_eater:
        if is_satisfied(eaters_to_recipes[new_loc], next_state.found_recipes):
            next_state.satisfied_eaters.add(new_loc)
            
    return next_state
    

def is_goal(state: State):
    if len(eaters_to_recipes) == len(state.satisfied_eaters):
            return True
    else:
        return False


# Breadth First Search (BFS)

### Pros:
As an uninformed search algorithm, has good time complexity.

Is optimal


### Cons:

Is slower than A*

Requires more space than IDS


### Implementation

States cant compared to each other normally. So we hash them and save them in visited states that is a set.

In each state we move our location to adjacents and after updating state properties, we add new states to frontier that keeps states sorted.

If a state is goal, it will be returned.


In [89]:
def BFS(file_address):
    visited_count = 0
    visited_states = set()

    frontier_heap = StateHeap(lambda state: state.time)
    frontier_heap.push(read_file(file_address))

    while frontier_heap:
        current_state = frontier_heap.pop()
        if current_state in visited_states:
            continue
        visited_states.add(current_state)
        
        cur_loc = current_state.loc
        for v in map.nodes[cur_loc].adj:
            new_state = move(current_state, v)
            visited_count += 1
            new_state.path.append(v + 1)

            if is_goal(new_state):
                    return (new_state, visited_count)
            frontier_heap.push(new_state)
            
    return (None, None)

`run` function runs a callable algorithm and returns `execution time`, `goal state` and `count of visited states`.

`test` function runs an algorithm 3 times and print optimal path, cost, average time and count of visited states.

In [90]:
def run(func: Callable, address: str) -> tuple[int, State, int] :
    tic=time.time()
    state, visited_count = func(address)
    toc=time.time()
    return toc-tic, state, visited_count

def test(func: Callable, address: str):
    time1, _, _ = run(func, address)
    time2, _, _ = run(func, address)
    time3, state, visited_count = run(func, address)

    path, cost = state.path, state.time
    average_time = (time1 + time2 + time3) / 3

    print(f"Path: {' -> '.join([str(node) for node in path])}")
    print(f"Cost: {cost}")
    print(f"Average time: {average_time}")
    print(f"Count of visited states: {visited_count}\n")


In [91]:
input1 = "testcases (easy)/input.txt"
input2 = "testcases (easy)/input2.txt"
input3 = "testcases (easy)/input3.txt"

test(BFS, input1)
test(BFS, input2)
test(BFS, input3)

Path: 1 -> 3 -> 4 -> 5 -> 7 -> 10 -> 11 -> 9 -> 8
Cost: 8
Average time: 0.010656038920084635
Count of visited states: 86

Path: 9 -> 10 -> 9 -> 4 -> 12 -> 3 -> 7 -> 5 -> 8
Cost: 8
Average time: 0.013332684834798178
Count of visited states: 182

Path: 13 -> 11 -> 10 -> 3 -> 2 -> 6 -> 12 -> 5 -> 9 -> 4 -> 1 -> 13 -> 11 -> 10
Cost: 13
Average time: 0.16677014032999674
Count of visited states: 2024



## Results
|Property|Test 1|Test 2|Test 3|
|:-:|:---:|:---:|:---:|
|**Time**          |0.010s|0.013s|0.166s|
|**Count of states**|86|182|2024|
|**Cost** |8|8|13|

&nbsp;
&nbsp;
&nbsp;
---

# Iterative Deepening Search (IDS)

### Pros:

Requires less memory than BFS and A*
Is optimal

### Cons:
Visits more states than BFS and A* and
It's possible to visit a state more than 1 time
Is Slower than BFS and A*

### Implementation:
We iterate over depth from 0 to as much as needed and call DFS with this limit.
DFS is a recursive function and is called for each adjacent of a node and new state would be created. When DFS through a subtree couldn't find a solution, after returning that state would be removed from visited states.

In [92]:
def DFS(current_state: State, visited_states: set[State], depth, visited_count=0):
    visited_states.add(current_state)

    if current_state.time > depth:
        return None, visited_count

    if is_goal(current_state):
        return (current_state, visited_count)

    cur_loc = current_state.loc
    for v in map.nodes[cur_loc].adj:
        new_state = move(current_state, v)
        if(new_state) in visited_states:
            continue
        new_state.path.append(v + 1)
        result, visited_count = DFS(new_state, visited_states, depth, visited_count+1)
        visited_states.remove(new_state)
        if result is not None:
            return result, visited_count

    return None, visited_count


def IDS(file_address):
    initial_state = read_file(file_address)
    depth = 0
    total_visited_count = 0
    while True:
        visited_states = set()
        result, visited_count = DFS(initial_state, visited_states, depth)
        total_visited_count += visited_count
        if result:
            return result, visited_count
        depth += 1


In [97]:
test(IDS, input1)
test(IDS, input2)
test(IDS, input3)

Path: 1 -> 3 -> 4 -> 5 -> 7 -> 10 -> 11 -> 9 -> 8
Cost: 8
Average time: 0.0940095583597819
Count of visited states: 232

Path: 9 -> 10 -> 9 -> 4 -> 12 -> 3 -> 7 -> 5 -> 8
Cost: 8
Average time: 0.24645423889160156
Count of visited states: 712

Path: 13 -> 11 -> 10 -> 3 -> 2 -> 6 -> 12 -> 5 -> 9 -> 4 -> 1 -> 13 -> 11 -> 10
Cost: 13
Average time: 3.6563045183817544
Count of visited states: 11406



## Results
|Property|Test 1|Test 2|Test 3|
|:-:|:---:|:---:|:---:|
|**Time**          |0.094s|0.246s|3.656s|
|**Count of states**|232|712|11406|
|**Cost** |8|8|13|

&nbsp;
&nbsp;
&nbsp;
---

# A*

### Pros:

Is optimal

Is informed search algorithm

Because of heuristic, is faster and visits less states than BFS and IDS


### Cons:

Finding heuristic functions may be difficult

Requires more space than IDS


### Implementation

It's exactly like BFS, but states in frontier will be sorted by `cost + heuristic(state)` and it can help us to check states with higher possiblity of success first. Heuristic clalculator is passed to `StateHeap` as lambda fucntion.


### Heuristic

**h(state) = count of nodes that have eater or recipe and hasn't been visited yet**

This heuristic is admissible because to reach goal state we should visit these special nodes at least. And is consistent because this case is true also between each 2 nodes.


In [68]:
def heuristic(state: State):
    h = 0
    for i in range(len(state.nodes)):
        if not state.nodes[i]['visited'] and map.nodes[i].is_special:
            h += 1
    return h
    

In [69]:
def A_star(file_address, alpha=1):
    total_visited_states = 0
    visited_states = set()

    frontier_heap = StateHeap(lambda state: state.time + alpha * heuristic(state))
    frontier_heap.push(read_file(file_address))

    while frontier_heap:
        current_state = frontier_heap.pop()
        if current_state in visited_states:
            continue
        visited_states.add(current_state)
        
        cur_loc = current_state.loc
        for v in map.nodes[cur_loc].adj:
            new_state = move(current_state, v)
            total_visited_states += 1
            new_state.path.append(v + 1)

            if is_goal(new_state):
                    return (new_state, total_visited_states)
            frontier_heap.push(new_state)
            
    return (None, None)   

# Weighted A*

### Implementation

For weighted A* we pass alpha to A* function. It can make heuristic unconsistent.


### Pros
Is faster than normal A*

### Cons
Is not optimal always.

In [98]:
print('--------alpha=1--------')
test(partial(A_star), input1)
test(partial(A_star), input2)
test(partial(A_star), input3)

--------alpha=1--------
Path: 1 -> 3 -> 4 -> 5 -> 7 -> 10 -> 11 -> 9 -> 8
Cost: 8
Average time: 0.010666290918986002
Count of visited states: 85

Path: 9 -> 10 -> 9 -> 4 -> 12 -> 3 -> 7 -> 5 -> 8
Cost: 8
Average time: 0.01708364486694336
Count of visited states: 165

Path: 13 -> 11 -> 10 -> 3 -> 2 -> 6 -> 12 -> 5 -> 9 -> 4 -> 1 -> 13 -> 11 -> 10
Cost: 13
Average time: 0.14486018816630045
Count of visited states: 1182



## Results
|Property|Test 1|Test 2|Test 3|
|:-:|:---:|:---:|:---:|
|**Time**          |0.0106s|0.017s|0.144s|
|**Count of states**|85|165|1182|
|**Cost** |8|8|13|

&nbsp;
&nbsp;
&nbsp;
---

In [99]:
print('--------alpha=1.4--------')
test(partial(A_star, alpha=1.4), input1)
test(partial(A_star, alpha=1.4), input2)
test(partial(A_star, alpha=1.4), input3)

--------alpha=1.4--------
Path: 1 -> 3 -> 4 -> 5 -> 4 -> 8 -> 9 -> 11 -> 9 -> 8
Cost: 9
Average time: 0.009000778198242188
Count of visited states: 76

Path: 9 -> 10 -> 9 -> 4 -> 12 -> 3 -> 7 -> 5 -> 8
Cost: 8
Average time: 0.013347784678141275
Count of visited states: 131

Path: 13 -> 11 -> 10 -> 3 -> 2 -> 6 -> 12 -> 5 -> 9 -> 4 -> 1 -> 13 -> 11 -> 10
Cost: 13
Average time: 0.06599887212117513
Count of visited states: 592



## Results
|Property|Test 1|Test 2|Test 3|
|:-:|:---:|:---:|:---:|
|**Time**          |0.009s|0.013s|0.065s|
|**Count of states**|76|131|592|
|**Cost** |9|8|13|

&nbsp;
&nbsp;
&nbsp;
---

In [100]:
print('--------alpha=3--------')
test(partial(A_star, alpha=3), input1)
test(partial(A_star, alpha=3), input2)
test(partial(A_star, alpha=3), input3)

--------alpha=3--------
Path: 1 -> 3 -> 4 -> 5 -> 4 -> 8 -> 9 -> 11 -> 9 -> 8
Cost: 9
Average time: 0.004664897918701172
Count of visited states: 33

Path: 9 -> 10 -> 8 -> 5 -> 7 -> 3 -> 12 -> 4 -> 2 -> 10 -> 8 -> 5 -> 7
Cost: 14
Average time: 0.007348060607910156
Count of visited states: 72

Path: 13 -> 1 -> 4 -> 9 -> 5 -> 10 -> 3 -> 2 -> 6 -> 12 -> 5 -> 10 -> 11 -> 13 -> 1 -> 4
Cost: 16
Average time: 0.026664177576700848
Count of visited states: 139



## Results
|Property|Test 1|Test 2|Test 3|
|:-:|:---:|:---:|:---:|
|**Time**          |0.004s|0.007s|0.026s|
|**Count of states**|33|72|139|
|**Cost** |9|14|16|

&nbsp;
&nbsp;
&nbsp;
---

# Overal Results

**- Test 1**

|**Method**|**Time**|**Cost**|**Count of states**|
|:-:|:---:|:---:|:---:|
|**BFS** |0.010s|8|86|
|**IDS** |0.094s|8|232|
|**A\*** |0.010s|8|85|
|**Weighted A\* ($\alpha = 1.4$)** |0.009s|9|76|
|**Weighted A\* ($\alpha = 3$)** |0.004s|9|33|

**- Test 2**

|**Method**|**Time**|**Cost**|**Count of states**|
|:-:|:---:|:---:|:---:|
|**BFS** |0.013s|8|182|
|**IDS** |0.246s|8|712|
|**A\*** |0.017s|8|165|
|**Weighted A\* ($\alpha = 1.4$)** |0.013s|8|131|
|**Weighted A\* ($\alpha = 3$)** |0.003s|14|72|

**- Test 3**

|**Method**|**Time**|**Cost**|**Count of states**|
|:-:|:---:|:---:|:---:|
|**BFS** |0.166s|13|2024|
|**IDS** |3.656s|13|11406|
|**A\*** |0.144s|13|1182|
|**Weighted A\* ($\alpha = 1.4$)** |0.065s|13|592|
|**Weighted A\* ($\alpha = 3$)** |0.026s|16|139|